In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***İmport Libraries***


In [ ]:
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
models = [LogisticRegression,KNeighborsClassifier,SVC,MLPClassifier,
            DecisionTreeClassifier,RandomForestClassifier,GradientBoostingClassifier,
            XGBClassifier,LGBMClassifier,CatBoostClassifier]

****Reading Datasets****

In [ ]:
lbe=LabelEncoder()
ord=OrdinalEncoder()
train=pd.read_csv("../input/titanic/train.csv")
test=pd.read_csv("../input/titanic/test.csv")
ornek=pd.read_csv("../input/titanic/gender_submission.csv")


**Null Value control**
****

In [ ]:
print(np.sort(train.isnull().sum()))
print(np.sort(test.isnull().sum()))

****Drop Train columns****


In [ ]:
train.drop(columns="Name",inplace=True)
train.drop(columns="Cabin",inplace=True)
train.drop(columns="Ticket",inplace=True)
train.drop(columns="PassengerId",inplace=True)

****Drop Test columns****

In [ ]:
test.drop(columns="Name",inplace=True)
test.drop(columns="Cabin",inplace=True)
test.drop(columns="Ticket",inplace=True)
test.drop(columns="PassengerId",inplace=True)

****Fill Na values for Train dataset****

In [ ]:
for i in ["Age","Embarked"]:
    train[i] = train.groupby(['Pclass', 'Sex'])[i].transform(lambda x: x.fillna(x.mode()[0]))

****Fill Na values for Test dataset****

In [ ]:
for i in ["Age","Embarked","Fare"]:
    test[i] = test.groupby(['Pclass', 'Sex'])[i].transform(lambda x: x.fillna(x.mode()[0]))

****Concating datasets****

In [ ]:
srv=train.Survived
train.drop(columns="Survived",inplace=True)
for i in train.columns:
    deneme=pd.concat([train[i],srv],axis=1)
    print(deneme.groupby(i).mean())
train=pd.concat([train,srv],axis=1)
print(np.sort(train.isnull().sum()))
print(np.sort(test.isnull().sum()))

****Train datasets labeling****

In [ ]:
"""train["Sex"].replace({"female":74,"male":19},inplace=True)
train["Pclass"].replace({1:63,2:47,3:24},inplace=True)
train["SibSp"].replace({0:35,1:54,2:46,3:25,4:16,5:0,8:0},inplace=True)
train["Parch"].replace({0:35,1:55,2:50,3:60,4:0,5:20,6:0},inplace=True)
train["Embarked"].replace({"C":55,"Q":39,"S":34},inplace=True)"""

Test datasets labeling

In [ ]:
"""test["Sex"].replace({"female":74,"male":19},inplace=True)
test["Pclass"].replace({1:63,2:47,3:24},inplace=True)
test["SibSp"].replace({0:35,1:54,2:46,3:25,4:16,5:0,8:0},inplace=True)
test["Parch"].replace({0:35,1:55,2:50,3:60,4:0,5:20,6:0},inplace=True)
test["Embarked"].replace({"C":55,"Q":39,"S":34},inplace=True)"""

In [ ]:
train_num=train.select_dtypes(include="number")
train_cat=train.select_dtypes(include="object")
test_num=test.select_dtypes(include="number")
test_cat=test.select_dtypes(include="object")
train_cat.columns

In [ ]:
for f in train_cat.columns:
    train_cat[f]=lbe.fit_transform(train_cat[f])
    
for f in test_cat.columns:
    test_cat[f]=lbe.fit_transform(test_cat[f])

    
data_train=pd.concat([train_num,train_cat],axis=1)
data_test=pd.concat([test_num,test_cat],axis=1)

****Cleaning Outliers***

In [ ]:
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(data_train)
df_scores = clf.negative_outlier_factor_
print(np.sort(df_scores)[0:30])


In [ ]:
esik_deger = np.sort(df_scores)[18]
aykiri_tf = df_scores > esik_deger
data_train=data_train[df_scores>esik_deger]

****Model Compare***

In [ ]:
def model_karsilastirma(df,model):
    X = df.drop(columns="Survived")
    X=scale(X)
    y = df["Survived"]

    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.15,
                                                        random_state=42)
    model_fit=model().fit(X_train,y_train)
    y_pred = model_fit.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(model ,"için sonuç doğruluk değeri:",acc)
    return acc

In [ ]:
for mod in models:
    model_karsilastirma(data_train,mod)

****Lgbm model Tuning****

In [ ]:
X = data_train.drop(columns="Survived")
X=scale(X)
y = data_train["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.15,
                                                    random_state=42)

lgbm = LGBMClassifier()
lgbm_params = {"learning_rate": [0.001, 0.01, 0.1],
              "n_estimators": [200, 500, 100],
              "max_depth":[1,2,35,8]}
lgbm_cv_model = GridSearchCV(lgbm,lgbm_params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)
lgbm_cv_model.best_params_

****LGBM Model****

In [ ]:
X = data_train.drop(columns="Survived")
X=scale(X)
y = data_train["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.15,
                                                    random_state=42)




lgbm_tuned = LGBMClassifier(learning_rate= 0.01,
                            max_depth= 35,
                            n_estimators= 200).fit(X_train, y_train)

y_pred = lgbm_tuned.predict(X_test)
acc=accuracy_score(y_test, y_pred)
print("LGBM: ",acc)
son_sonuc=ornek
son_sonuc.Survived=lgbm_tuned.predict(scale(data_test))
son_sonuc.to_csv("Titanic_Test3.csv",index=False)